# Spelling Corrector : Pulaar
In this notebook, we will implement an auto-correct system using pulaar language. Pulaar is spoken by million people across about 20 countries in West and Central Africa. This notebook is inspired by [Peter Norvig](https://en.wikipedia.org/wiki/Peter_Norvig) who first created auto-correct in 2007 and [Deeplearning.ai](https://www.deeplearning.ai/program/natural-language-processing-specialization/) NLP specialization I took last year. Below the [original article](https://norvig.com/spell-correct.html) of Peter Norvig.

Auto-correct systems are nowadays everywhere. 
- For example, if you type in the word **"atocorec"**, chances are very high that you meant to write **"autocorrect"**, as shown in **Figure 1**. 

<div style="width:image width px; font-size:100%; text-align:center;"><img src='https://cdn.techterms.com/img/md/autocorrect_1317.png' alt="alternate text" width="width" height="height" style="width:300px;height:250px;" /> Figure 1 </div>

<a name='0-1'></a>
#### Edit Distance

We will implement models that correct words that are 1 and 2 edit distances away. 
- We say two words are n edit distance away from each other when we need n edits to change one word into another. 

An edit could consist of one of the following options: 

- Delete (remove a letter): ‘hat’ => ‘at, ha, ht’
- Switch (swap 2 adjacent letters): ‘eta’ => ‘eat, tea,...’
- Replace (change 1 letter to another): ‘jat’ => ‘hat, rat, cat, mat, ...’
- Insert (add a letter): ‘te’ => ‘the, ten, ate, ...’

We will be using the four methods above to implement an Auto-correct. 

- To do so, we will need to compute probabilities that a certain word is correct given an input. 

The goal of our spell check model is to compute the following probability:

\begin{equation}
P(c|w) = \frac{P(w|c)\times P(c)}{P(w)} \tag{Eqn-1}
\end{equation}

The equation above is [Bayes Rule](https://en.wikipedia.org/wiki/Bayes%27_theorem). 
- Equation 1 says that the probability of a word being correct $P(c|w) $is equal to the probability of having a certain word $w$, given that it is correct $P(w|c)$, multiplied by the probability of being correct in general $P(C)$ divided by the probability of that word $w$ appearing $P(w)$ in general.
- To compute equation 1, we will first import a data set and then create all the probabilities that we need using that data set. 

<a name='1'></a>
# Part 1: Data Preprocessing 

In [1]:
import re
from collections import Counter
import numpy as np
import pandas as pd

As in any other machine learning task, the first thing we have to do is process our data set. Let's write a function that reads our our text file, lowercase everything and returns a list of words.

In [2]:
def process_data(file_name):
    """
    Input: 
        A file_name  
    Output: 
        words: a list containing all the words in the text file in lower case. 
    """
    words = [] 
    file = open(file_name).read()
    words = re.findall(r'\w+', file.lower())
    return words

In [3]:
word_list = process_data('pulaar.txt')

Note, in the following cell, 'words' is converted to a python `set`. This eliminates any duplicate entries.

In [4]:
vocab = set(word_list)  # this will be your new vocabulary
print(f"The first ten words in the text are: \n{word_list[0:10]}")
print(f"There are {len(vocab)} unique words in the vocabulary.")

The first ten words in the text are: 
['hitaande', 'waɗi', 'ko', 'lebbi', 'sappo', 'e', 'ɗiɗi', 'mi', 'yiɗaa', 'ñaameele']
There are 3705 unique words in the vocabulary.


In the following cell, we will compute a `get_count` function that returns a dictionary
- The dictionary's keys are words
- The value for each word is the number of times that word appears in the corpus. 

For example, given the following sentence: **"I am happy because I am learning"**, the dictionary should return the following: 
<table style="width:20%">

  <tr>
    <td> <b>Key </b>  </td>
    <td> <b>Value </b> </td> 


  </tr>
  <tr>
    <td> I  </td>
    <td> 2</td> 
 
  </tr>
   
  <tr>
    <td>am</td>
    <td>2</td> 
  </tr>

  <tr>
    <td>happy</td>
    <td>1</td> 
  </tr>
  
   <tr>
    <td>because</td>
    <td>1</td> 
  </tr>
  
   <tr>
    <td>learning</td>
    <td>1</td> 
  </tr>
</table>



#### Word count

In [5]:
def get_count(word_l):
    '''
    Input:
        word_l: a set of words representing the corpus. 
    Output:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    '''   
    word_count_dict = {}  
    word_count_dict = Counter(word_l)
    return word_count_dict

In [6]:
word_count_dict = get_count(word_list)
print(f"There are {len(word_count_dict)} key values pairs")
print(f"The count for the word 'hitaande' is {word_count_dict.get('hitaande',0)}")

There are 3705 key values pairs
The count for the word 'hitaande' is 5


#### Probability 
Given the dictionary of word counts, we will compute the probability that each word will appear if randomly selected from the corpus of words.

$$P(w_i) = \frac{C(w_i)}{M} \tag{Eqn-2}$$
where 

$C(w_i)$ is the total number of times $w_i$ appears in the corpus.

$M$ is the total number of words in the corpus.

For example, the probability of the word 'am' in the sentence **'I am happy because I am learning'** is:

$$P(am) = \frac{C(w_i)}{M} = \frac {2}{7} \tag{Eqn-3}.$$


In [7]:
def get_probs(word_count_dict):
    '''
    Input:
        word_count_dict: The wordcount dictionary where key is the word and value is its frequency.
    Output:
        probs: A dictionary where keys are the words and the values are the probability that a word will occur. 
    '''
    probs = {} 
    probs = {word: word_count_dict[word]/sum(word_count_dict.values()) for word in word_count_dict}
    return probs

In [8]:
probs = get_probs(word_count_dict)
print(f"Length of probs is {len(probs)}")
print(f"P('jaŋde') is {probs['jaŋde']:.4f}")

Length of probs is 3705
P('jaŋde') is 0.0022


# Part 2: String Manipulations

Now, that we have computed $P(w_i)$ for all the words in the corpus, we will write a few functions to manipulate strings so that we can edit the erroneous strings and return the right spellings of the words. In this section, we will implement four functions: 

* `delete_letter`: given a word, it returns all the possible strings that have **one character removed**. 
* `switch_letter`: given a word, it returns all the possible strings that have **two adjacent letters switched**.
* `replace_letter`: given a word, it returns all the possible strings that have **one character replaced by another different letter**.
* `insert_letter`: given a word, it returns all the possible strings that have an **additional character inserted**. 


#### Delete letter 

Let's implement a `delete_letter()` function that, given a word, returns a list of strings with one character deleted. 

For example, given the word **jaŋde**, it would return the set: {'aŋde', 'jŋde', 'jade', 'jaŋe', 'jaŋd'}. 


In [9]:
def delete_letter(word, verbose=False):
    '''
    Input:
        word: the string/word for which we will generate all possible words 
                in the vocabulary which have 1 missing character
    Output:
        delete_l: a list of all possible strings obtained by deleting 1 character from word
    '''
    
    split_l = [(word[:i],word[i:]) for i in range(len(word) + 1)]
    delete_l = [L + R[1:] for L, R in split_l if R]

    if verbose: print(f"Input word {word}, \nsplit_l = {split_l}, \ndelete_l = {delete_l}")

    return delete_l

In [10]:
delete_word_l = delete_letter(word="jaŋde",
                        verbose=True)

Input word jaŋde, 
split_l = [('', 'jaŋde'), ('j', 'aŋde'), ('ja', 'ŋde'), ('jaŋ', 'de'), ('jaŋd', 'e'), ('jaŋde', '')], 
delete_l = ['aŋde', 'jŋde', 'jade', 'jaŋe', 'jaŋd']


<a name='ex-5'></a>
#### Switch letter 

Now we will implement a function that switches two letters in a word. It takes in a word and returns a list of all the possible switches of two letters **that are adjacent to each other**. 
- For example, given the word 'eta', it returns {'eat', 'tea'}, but does not return 'ate'.


In [11]:
def switch_letter(word, verbose=False):
    '''
    Input:
        word: input string
     Output:
        switches: a list of all possible strings with one adjacent charater switched
    ''' 
    
    switch_l = []
    split_l = []
    
    split_l = [(word[:i],word[i:]) for i in range(len(word) + 1)]
    switch_l = [L + R[1] + R[0] + R[2:] for L, R in split_l if len(R)>1]
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nswitch_l = {switch_l}") 

    return switch_l

In [12]:
switch_word_l = switch_letter(word="jaŋde",
                         verbose=True)

Input word = jaŋde 
split_l = [('', 'jaŋde'), ('j', 'aŋde'), ('ja', 'ŋde'), ('jaŋ', 'de'), ('jaŋd', 'e'), ('jaŋde', '')] 
switch_l = ['ajŋde', 'jŋade', 'jadŋe', 'jaŋed']


#### Replace letter
Let's implement a function that takes in a word and returns a list of strings with one **replaced letter** from the original word. 

In [15]:
def replace_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        replaces: a list of all possible strings where we replaced one letter from the original word. 
    ''' 
    
    letters = 'abɓcdɗefghijklmnŋñoprstuwyƴz'
    
    split_l = [(word[:i],word[i:]) for i in range(len(word) + 1)]
    replace_l = [L + c + R[1:] for L, R in split_l if R for c in letters]

    replace_set = set(replace_l)
    replace_l = sorted(list(replace_set))
    
    if verbose: print(f"Input word = {word} \nsplit_l = {split_l} \nreplace_l {replace_l}")   
    
    return replace_l

In [16]:
replace_l = replace_letter(word='jaŋde',
                              verbose=True)

Input word = jaŋde 
split_l = [('', 'jaŋde'), ('j', 'aŋde'), ('ja', 'ŋde'), ('jaŋ', 'de'), ('jaŋd', 'e'), ('jaŋde', '')] 
replace_l ['aaŋde', 'baŋde', 'caŋde', 'daŋde', 'eaŋde', 'faŋde', 'gaŋde', 'haŋde', 'iaŋde', 'jaade', 'jabde', 'jacde', 'jadde', 'jaede', 'jafde', 'jagde', 'jahde', 'jaide', 'jajde', 'jakde', 'jalde', 'jamde', 'jande', 'jaode', 'japde', 'jarde', 'jasde', 'jatde', 'jaude', 'jawde', 'jayde', 'jazde', 'jañde', 'jaŋae', 'jaŋbe', 'jaŋce', 'jaŋda', 'jaŋdb', 'jaŋdc', 'jaŋdd', 'jaŋde', 'jaŋdf', 'jaŋdg', 'jaŋdh', 'jaŋdi', 'jaŋdj', 'jaŋdk', 'jaŋdl', 'jaŋdm', 'jaŋdn', 'jaŋdo', 'jaŋdp', 'jaŋdr', 'jaŋds', 'jaŋdt', 'jaŋdu', 'jaŋdw', 'jaŋdy', 'jaŋdz', 'jaŋdñ', 'jaŋdŋ', 'jaŋdƴ', 'jaŋdɓ', 'jaŋdɗ', 'jaŋee', 'jaŋfe', 'jaŋge', 'jaŋhe', 'jaŋie', 'jaŋje', 'jaŋke', 'jaŋle', 'jaŋme', 'jaŋne', 'jaŋoe', 'jaŋpe', 'jaŋre', 'jaŋse', 'jaŋte', 'jaŋue', 'jaŋwe', 'jaŋye', 'jaŋze', 'jaŋñe', 'jaŋŋe', 'jaŋƴe', 'jaŋɓe', 'jaŋɗe', 'jaƴde', 'jaɓde', 'jaɗde', 'jbŋde', 'jcŋde', 'jdŋde', 'jeŋde', 'jfŋde', 'jg

#### Insert letter 

In the next cell, we will compute a function that takes in a word and returns a list with a letter inserted at every offset.  

In [17]:
def insert_letter(word, verbose=False):
    '''
    Input:
        word: the input string/word 
    Output:
        inserts: a set of all possible strings with one new letter inserted at every offset
    ''' 
    letters = 'abɓcdɗefghijklmnŋñoprstuwyƴz'
    
    split_l = [(word[:i],word[i:]) for i in range(len(word) + 1)]
    insert_l = [L + c + R for L, R in split_l for c in letters]

    if verbose: print(f"Input word {word} \nsplit_l = {split_l} \ninsert_l = {insert_l}")
    
    return insert_l

In [18]:
insert_l = insert_letter('jaŋde', True)
print(f"Number of strings output by insert_letter('jaŋde') is {len(insert_l)}")

Input word jaŋde 
split_l = [('', 'jaŋde'), ('j', 'aŋde'), ('ja', 'ŋde'), ('jaŋ', 'de'), ('jaŋd', 'e'), ('jaŋde', '')] 
insert_l = ['ajaŋde', 'bjaŋde', 'ɓjaŋde', 'cjaŋde', 'djaŋde', 'ɗjaŋde', 'ejaŋde', 'fjaŋde', 'gjaŋde', 'hjaŋde', 'ijaŋde', 'jjaŋde', 'kjaŋde', 'ljaŋde', 'mjaŋde', 'njaŋde', 'ŋjaŋde', 'ñjaŋde', 'ojaŋde', 'pjaŋde', 'rjaŋde', 'sjaŋde', 'tjaŋde', 'ujaŋde', 'wjaŋde', 'yjaŋde', 'ƴjaŋde', 'zjaŋde', 'jaaŋde', 'jbaŋde', 'jɓaŋde', 'jcaŋde', 'jdaŋde', 'jɗaŋde', 'jeaŋde', 'jfaŋde', 'jgaŋde', 'jhaŋde', 'jiaŋde', 'jjaŋde', 'jkaŋde', 'jlaŋde', 'jmaŋde', 'jnaŋde', 'jŋaŋde', 'jñaŋde', 'joaŋde', 'jpaŋde', 'jraŋde', 'jsaŋde', 'jtaŋde', 'juaŋde', 'jwaŋde', 'jyaŋde', 'jƴaŋde', 'jzaŋde', 'jaaŋde', 'jabŋde', 'jaɓŋde', 'jacŋde', 'jadŋde', 'jaɗŋde', 'jaeŋde', 'jafŋde', 'jagŋde', 'jahŋde', 'jaiŋde', 'jajŋde', 'jakŋde', 'jalŋde', 'jamŋde', 'janŋde', 'jaŋŋde', 'jañŋde', 'jaoŋde', 'japŋde', 'jarŋde', 'jasŋde', 'jatŋde', 'jauŋde', 'jawŋde', 'jayŋde', 'jaƴŋde', 'jazŋde', 'jaŋade', 'jaŋbde', 'jaŋɓde'

# Part 3: Combining the edits

Now that we have implemented the string manipulations, we will create two functions that, given a string, will return all the possible single and double edits on that string. These will be `edit_one_letter()` and `edit_two_letters()`.

## 3.1 Edit one letter


Let's implement the `edit_one_letter` function to get all the possible edits that are one edit away from a word. The edits  consist of the replace, insert, delete, and optionally the switch operation. We will use the previous functions we have already implemented to complete this function. The 'switch' function  is a less common edit function, so its use will be selected by an "allow_switches" input argument.

Note that those functions return *lists* while this function should return a *python set*. Utilizing a set eliminates any duplicate entries.

In [19]:
def edit_one_letter(word, allow_switches = True):
    """
    Input:
        word: the string/word for which we will generate all possible wordsthat are one edit away.
    Output:
        edit_one_set: a set of words with one possible edit. Please return a set. and not a list.
    """
    
    edit_one_set = set()
        
    edit_one_set.update(delete_letter(word))
    if allow_switches:
        edit_one_set.update(switch_letter(word))
    edit_one_set.update(replace_letter(word))
    edit_one_set.update(insert_letter(word))

    return edit_one_set

In [20]:
tmp_word = "jaŋde"
tmp_edit_one_set = edit_one_letter(tmp_word)
# turn this into a list to sort it, in order to view it
tmp_edit_one_l = sorted(list(tmp_edit_one_set))

print(f"input word {tmp_word} \nedit_one_l \n{tmp_edit_one_l}\n")
print(f"The type of the returned object should be a set {type(tmp_edit_one_set)}")
print(f"Number of outputs from edit_one_letter('jaŋde') is {len(edit_one_letter('jaŋde'))}")

input word jaŋde 
edit_one_l 
['aaŋde', 'ajaŋde', 'ajŋde', 'aŋde', 'baŋde', 'bjaŋde', 'caŋde', 'cjaŋde', 'daŋde', 'djaŋde', 'eaŋde', 'ejaŋde', 'faŋde', 'fjaŋde', 'gaŋde', 'gjaŋde', 'haŋde', 'hjaŋde', 'iaŋde', 'ijaŋde', 'jaade', 'jaaŋde', 'jabde', 'jabŋde', 'jacde', 'jacŋde', 'jadde', 'jade', 'jadŋde', 'jadŋe', 'jaede', 'jaeŋde', 'jafde', 'jafŋde', 'jagde', 'jagŋde', 'jahde', 'jahŋde', 'jaide', 'jaiŋde', 'jajde', 'jajŋde', 'jakde', 'jakŋde', 'jalde', 'jalŋde', 'jamde', 'jamŋde', 'jande', 'janŋde', 'jaode', 'jaoŋde', 'japde', 'japŋde', 'jarde', 'jarŋde', 'jasde', 'jasŋde', 'jatde', 'jatŋde', 'jaude', 'jauŋde', 'jawde', 'jawŋde', 'jayde', 'jayŋde', 'jazde', 'jazŋde', 'jañde', 'jañŋde', 'jaŋade', 'jaŋae', 'jaŋbde', 'jaŋbe', 'jaŋcde', 'jaŋce', 'jaŋd', 'jaŋda', 'jaŋdae', 'jaŋdb', 'jaŋdbe', 'jaŋdc', 'jaŋdce', 'jaŋdd', 'jaŋdde', 'jaŋde', 'jaŋdea', 'jaŋdeb', 'jaŋdec', 'jaŋded', 'jaŋdee', 'jaŋdef', 'jaŋdeg', 'jaŋdeh', 'jaŋdei', 'jaŋdej', 'jaŋdek', 'jaŋdel', 'jaŋdem', 'jaŋden', 'jaŋdeo', 'jaŋdep'

## Part 3.2 Edit two letters



Let's implement two edits on a word function. To do so, we will get all the possible edits on a single word and then for each modified word, we will modify it again. 


In [22]:
def edit_two_letters(word, allow_switches = True):
    '''
    Input:
        word: the input string/word 
    Output:
        edit_two_set: a set of strings with all possible two edits
    '''
    
        
    edit_two_set = list(e2 for e1 in edit_one_letter(word) for e2 in edit_one_letter(e1))
    edit_two_set = set(edit_two_set)
    
    return edit_two_set

In [24]:
tmp_edit_two_set = edit_two_letters("mi")
tmp_edit_two_l = sorted(list(tmp_edit_two_set))
print(f"Number of strings with edit distance of two: {len(tmp_edit_two_l)}")
print(f"First 10 strings {tmp_edit_two_l[:10]}")
print(f"Last 10 strings {tmp_edit_two_l[-10:]}")
print(f"The data type of the returned object should be a set {type(tmp_edit_two_set)}")
print(f"Number of strings that are 2 edit distances from 'mi' is {len(edit_two_letters('mi'))}")

Number of strings with edit distance of two: 8320
First 10 strings ['', 'a', 'aa', 'aai', 'aami', 'ab', 'abi', 'abmi', 'ac', 'aci']
Last 10 strings ['ɗŋmi', 'ɗƴ', 'ɗƴi', 'ɗƴmi', 'ɗɓ', 'ɗɓi', 'ɗɓmi', 'ɗɗ', 'ɗɗi', 'ɗɗmi']
The data type of the returned object should be a set <class 'set'>
Number of strings that are 2 edit distances from 'mi' is 8320


<a name='3-3'></a>
## Part 3-3:  Spelling suggestions

Now you that we have computed `edit_two_letters` function, we will use it to get a set of all the possible 2 edits on the input word. We will then use those strings to get the most probable word we meant to type i.e the typing suggestion.

Let's implement `get_corrections`, which returns a list of zero to n possible suggestion tuples of the form (word, probability_of_word). 

**Step 1:** Generate suggestions for a supplied word: We'll use the edit functions we have developed. The 'suggestion algorithm' should follow this logic: 
* If the word is in the vocabulary, suggest the word. 
* Otherwise, if there are suggestions from `edit_one_letter` that are in the vocabulary, use those. 
* Otherwise, if there are suggestions from `edit_two_letters` that are in the vocabulary, use those. 
* Otherwise, suggest the input word.*  
* The idea is that words generated from fewer edits are more likely than words with more edits.


**Step 2**: Create a 'best_words' dictionary where the 'key' is a suggestion and the 'value' is the probability of that word in the vocabulary. If the word is not in the vocabulary, assign it a probability of 0.

**Step 3**: Select the n best suggestions. There may be fewer than n.

In [26]:
def get_corrections(word, probs, vocab, n=2, verbose = False):
    '''
    Input: 
        word: a user entered string to check for suggestions
        probs: a dictionary that maps each word to its probability in the corpus
        vocab: a set containing all the vocabulary
        n: number of possible word corrections you want returned in the dictionary
    Output: 
        n_best: a list of tuples with the most probable n corrected words and their probabilities.
    '''  
    suggestions = []
    n_best = []
    suggestions = list((word in vocab and word) or edit_one_letter(word).intersection(vocab) or edit_two_letters(word).intersection(vocab))
    n_best = [[s,probs[s]] for s in list(reversed(suggestions))]
    
    if verbose: print("suggestions = ", suggestions)

    return n_best

In [45]:
def auto_correct(input_word):
    correction = get_corrections(input_word, probs, vocab, 2, verbose=True)
    for i, word_prob in enumerate(correction):
        print(f"word {i}: {word_prob[0]}, probability {word_prob[1]:.6f}")

#### Let's test our implementation with the word `mbaroodi` which means `lion` in pulaar. We will purposely omit one letter in the word. Let's say the `o`.

In [48]:
auto_correct('mbarodi')

suggestions =  ['mbaroodi']
word 0: mbaroodi, probability 0.000222


#### Another example with the word  `puccu/pucci` which means `horse/horses` in pulaar.

In [49]:
auto_correct('pucc')

suggestions =  ['pucci', 'puccu']
word 0: puccu, probability 0.000296
word 1: pucci, probability 0.000222


#### Another example with the `bottaari` which means `lunch` in pulaar.

In [52]:
auto_correct('bottary')

suggestions =  ['bottaari']
word 0: bottaari, probability 0.000222


#### An example with the word `ñaagaade` which means `beg/ask` in pulaar.

In [53]:
auto_correct('ñagoade')

suggestions =  ['ñaagaade']
word 0: ñaagaade, probability 0.000148


#### An example with the word `ñemmbiɗinde` which means `model/modeling` in pulaar.

In [59]:
auto_correct('ñemmbiɗing')

suggestions =  ['ñemmbiɗingol', 'ñemmbiɗinde']
word 0: ñemmbiɗinde, probability 0.000148
word 1: ñemmbiɗingol, probability 0.000148


# Part 4: Minimum Edit distance

Now that we have implemented the auto-correct, how do we evaluate the similarity between two strings? 
How do you efficiently find the shortest path to go from the word, 'waht' for instance to the word 'what'?

We will implement a dynamic programming system that will tell us the minimum number of edits required to convert a string into another string. To learn more about Dynamic programming, check out this lecture from [stanford](https://web.stanford.edu/class/cs124/lec/med.pdf)

### Part 4.1 Dynamic Programming

Let's implement the function below to get the minimum amount of edits required given a source string and a target string. 

In [31]:
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    '''
    Input: 
        source: a string corresponding to the string you are starting with
        target: a string corresponding to the string you want to end with
        ins_cost: an integer setting the insert cost
        del_cost: an integer setting the delete cost
        rep_cost: an integer setting the replace cost
    Output:
        D: a matrix of len(source)+1 by len(target)+1 containing minimum edit distances
        med: the minimum edit distance (med) required to convert the source string to the target
    '''
    # use deletion and insert cost as  1
    m = len(source) 
    n = len(target) 
    
    #initialize cost matrix with zeros and dimensions (m+1,n+1) 
    D = np.zeros((m+1, n+1), dtype=int) 
    
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    
    # Fill in column 0, from row 1 to row m, both inclusive
    for row in range(1,m+1): 
        D[row,0] = D[row - 1, 0] + del_cost
        
    # Fill in row 0, for all columns from 1 to n, both inclusive
    for col in range(1,n+1): 
        D[0,col] = D[0,col - 1] + ins_cost
        
    # Loop through row 1 to row m, both inclusive
    for row in range(1,m+1): 
        
        # Loop through column 1 to column n, both inclusive
        for col in range(1,n+1):
            
            # Intialize r_cost to the 'replace' cost that is passed into this function
            r_cost = rep_cost
            
            # Check to see if source character at the previous row
            # matches the target character at the previous column, 
            if source[row - 1] == target[col - 1]:
                
                # Update the replacement cost to 0 if source and target are the same
                r_cost = 0
                
            # Update the cost at row, col based on previous entries in the cost matrix
            # Refer to the equation calculate for D[i,j] (the minimum of three calculated costs)
            D[row,col] = min(D[row-1,col] + del_cost, D[row, col-1] + ins_cost, D[row-1,col-1] + r_cost)
          
    # Set the minimum edit distance with the cost found at row m, column n
    med = D[m,n]
    
    return D, med

In [32]:
# testing your implementation 
source =  'ngaaro'
target = 'ngaari'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list('#' + source)
cols = list('#' + target)
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  2 

   #  n  g  a  a  r  i
#  0  1  2  3  4  5  6
n  1  0  1  2  3  4  5
g  2  1  0  1  2  3  4
a  3  2  1  0  1  2  3
a  4  3  2  1  0  1  2
r  5  4  3  2  1  0  1
o  6  5  4  3  2  1  2


In [33]:
#DO NOT MODIFY THIS CELL
# testing your implementation 
source =  'guto'
target = 'gite'
matrix, min_edits = min_edit_distance(source, target)
print("minimum edits: ",min_edits, "\n")
idx = list(source)
idx.insert(0, '#')
cols = list(target)
cols.insert(0, '#')
df = pd.DataFrame(matrix, index=idx, columns= cols)
print(df)

minimum edits:  4 

   #  g  i  t  e
#  0  1  2  3  4
g  1  0  1  2  3
u  2  1  2  3  4
t  3  2  3  2  3
o  4  3  4  3  4
